In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
import glob

import pandas as pd
import joblib

def dumpJoblib(fileName, obj):
    with open(fileName, mode="wb") as f:
        joblib.dump(obj, f, compress=3)
        
def loadJoblib(fileName):
    with open(fileName, mode="rb") as f:
        return joblib.load(f)
    
df = loadJoblib('../Data/contain.joblib')
df.head()

,tweet_id,tweet_full_text,finlabel
0,1370151453805871104,おはようございます☀\n\n休職382日目。昨日はたくさんのリプありがとうございました😭✨こ...,1
1,1396246713044652032,就活生のためのグループディスカッション対策まとめ(40176view)｜ https://t...,4
2,1397335585459560449,【登録してくべき就活サイト】\n就職活動において、情報量が大きな武器になります。 以下のサイ...,4
3,1375959601900777474,[就活のお悩みを記入してね] #宇賀なつみ アナに就活相談しよう！抽選で8名様に宇賀アナへの...,3
4,1347685399552880641,ESは就活におけるあなたの名刺代わりのようなもの、\n面接でもベースになるのはES、甘く見て...,4


In [14]:
data = list(df['tweet_id'])
label = list(df['finlabel'])

In [15]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

     |████████████████████████████████| 2.1 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 282 kB 41.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.4 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 830 kB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 45.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 46.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 30.5 MB/s eta 0:00:01
     |████████████████████████████████| 329 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 147.1 MB 5.5 kB/s eta 0:00:011    |██████████████████████▊         | 104.5 MB 9.3 MB/s eta 0:00:05     |███████████████████████████▏    | 124.8 MB 8.9 MB/s eta 0:00:03
     |████████████████████████████████| 155 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 31.1 MB/s eta 0:00:01
     |███████████████████████████

In [16]:
import random 
import glob
import json
from tqdm import tqdm
import unicodedata

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel

from RESEARCH-jobhunting-datasetsPY_classification.multi_classification import BertForSequenceClassificationMultiLabel

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

ModuleNotFoundError: No module named 'PY_classification'

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
bert_scml = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=4
) 
bert_scml = bert_scml.to(device=device)

In [7]:
labels = []
for emotion in label:
    if emotion == 63:
        labels.append([1, 0, 0, 0])
    elif emotion == 64:
        labels.append([0, 1, 0, 0])
    elif emotion == 65:
        labels.append([0, 0, 1, 0])
    else:
        labels.append([0, 0, 0, 1])

In [8]:
max_length = 152
dataset_for_loader = []

for text, label in zip(data, labels):
    encoding = tokenizer(
      text, 
      max_length=max_length,
      padding='max_length',
      truncation=True
    )
    encoding['labels'] = label
    encoding = {k: torch.tensor(v).to(device=device) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
    

random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

In [9]:
dataloader_train = DataLoader(
    dataset_train, batch_size=16, shuffle=True
)
dataloader_val = DataLoader(
    dataset_val, batch_size=16
)
dataloader_test = DataLoader(
    dataset_test, batch_size=16
)

dataloader_val

In [12]:
from PY_classification.pytorch_lightning_multi import BertForSequenceClassificationMultiLabel_pl

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='part/model/'
)

#学習方法指定
trainer = pl.Trainer(
    gpus=1,
    max_epochs=10,
    callbacks=[checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, num_labels=4, lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                                    | Params
----------------------------------------------------------------------
0 | bert_scml | BertForSequenceClassificationMultiLabel | 110 M 
----------------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.482   Total estimated model params size (MB)
/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [17]:
best_model_path = checkpoint.best_model_path
print(f'validation data loss:{checkpoint.best_model_score}')
print(f'The best path is {best_model_path}')

validation data loss:0.2354281097650528
The best path is /home/takakiyuto/Desktop/RESEARCH-COVID19-datasets/part/model/epoch=0-step=1406.ckpt


In [14]:
%load_ext tensorboard
%tensorboard --logdir ./

In [15]:
test = trainer.test(test_dataloaders=dataloader_test, ckpt_path=best_model_path)
print(f'Accuracy: {test[0]["accuracy"]:.4f}')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.789712131023407}
--------------------------------------------------------------------------------
Accuracy: 0.7897
